In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle

from model import build_transformer
from config import get_config
from train import train_model
from transformer_dataset import TSDataset, causal_mask, prepare_time_series_data, scale_data_seq, create_lags

from sklearn.preprocessing import StandardScaler

In [2]:
optimus = build_transformer(d_model=48)
batch_size = 24

num_features = 48
in_seq = 32
out_seq = 8

X = torch.rand(batch_size, in_seq, num_features)
y = torch.rand(batch_size, out_seq, 1)

In [13]:
X.shape

torch.Size([24, 32, 48])

In [14]:
tgt_mask = causal_mask(8)
src_mask = torch.ones(1, in_seq, in_seq).bool()

In [15]:
encoder_output = optimus.encode(X, src_mask)
decoder_output = optimus.decode(encoder_output, src_mask, y, tgt_mask)
proj_output = optimus.project(decoder_output)
## PIPELINE WORKS, THATS GOOD

RuntimeError: mat1 and mat2 shapes cannot be multiplied (768x48 and 58x58)

In [ ]:
config = get_config()
device = "cpu"

In [10]:
from torch.utils.data import DataLoader
import os

with open(os.path.join(config["path_pickle"], config["data_pickle_name"]), 'rb') as file:
    ds_raw =  pickle.load(file)
ds_lags, new_vars = create_lags(ds_raw, config["lags"], config["diffs"])
# add new lagged variables to the exo_vars in config
config["exo_vars"] += new_vars

# train-val split
training_split = pd.Timestamp('2023-04-01 00:00:00')
data_train = ds_lags[lambda x: x.datetime_utc <= training_split].copy()
data_train = data_train.fillna(data_train.median(numeric_only=True))

data_val = ds_lags[lambda x: x.datetime_utc > training_split].copy()
data_val = data_val.fillna(data_train.median(numeric_only=True))

train_data_raw, train_data_tensor, train_label_tensor = prepare_time_series_data(data_train, config["exo_vars"], config["target"], config['src_seq_len'], config['tgt_seq_len'])
val_data_raw, _, _ = prepare_time_series_data(data_val, config["exo_vars"], config["target"], config['src_seq_len'], config['tgt_seq_len'])


data_to_scale = {
    "train": train_data_raw,
    "val": val_data_raw,
}

data_scaled, label_scaler = scale_data_seq(train_data_tensor, train_label_tensor, data_to_scale)
train_scl, val_scl = data_scaled

train_ds = TSDataset(train_scl, config['src_seq_len'], config['tgt_seq_len'])
val_ds = TSDataset(val_scl, config['src_seq_len'], config['tgt_seq_len'])

train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=config['batch_size']*10, shuffle=False)
val_dataloader_onebatch = DataLoader(val_ds, batch_size=1, shuffle=True)

In [7]:
import os
data_TS = "../../data/data_TS/"

In [8]:
with open(os.path.join(data_TS, 'merged_data.pkl'), 'rb') as file:
    data =  pickle.load(file)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58360 entries, 4 to 58363
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   datetime_utc                  58360 non-null  datetime64[ns]
 1   holiday                       58360 non-null  float64       
 2   not_working                   58360 non-null  float64       
 3   net_regulation_volume         58360 non-null  float64       
 4   system_imbalance              58360 non-null  float64       
 5   alpha                         58360 non-null  float64       
 6   marginal_incremental_price    58360 non-null  float64       
 7   marginal_decremental_price    58360 non-null  float64       
 8   positive_imbalance_price      58360 non-null  float64       
 9   negative_imbalance_price      58360 non-null  float64       
 10  measured_&_upscaled_solar     58360 non-null  float64       
 11  most_recent_forecast_solar  

In [10]:
config['lags'] = {
            "system_imbalance": [i for i in range(4*24-2*4, 4*24+6*4)] + [i for i in range(7*4*24-2*4, 7*4*24+6*4)]
            }

In [11]:
data.shape

(58360, 40)

In [12]:
ds_lags, new_vars = create_lags(data, config["lags"], config["diffs"])
config["exo_vars"] += new_vars

In [13]:
training_split = pd.Timestamp('2023-04-01 00:00:00')

data_train = ds_lags[lambda x: x.datetime_utc <= training_split].copy()
data_train = data_train.fillna(data_train.median(numeric_only=True))

data_val = ds_lags[lambda x: x.datetime_utc > training_split].copy()
data_val = data_val.fillna(data_train.median(numeric_only=True))

print("Train:", len(data_train))
print("Val:", len(data_val))

Train: 42986
Val: 14679


In [19]:
train_data_raw, train_data_tensor, train_label_tensor = prepare_time_series_data(data_train, config["exo_vars"], config["target"], config['src_seq_len'], config['tgt_seq_len'])
val_data_raw, _, _ = prepare_time_series_data(data_val, config["exo_vars"], config["target"], config['src_seq_len'], config['tgt_seq_len'])


In [26]:
data_to_scale = {
    "train": train_data_raw,
    "val": val_data_raw,
}

In [27]:
data_scaled, label_scaler = scale_data_seq(train_data_tensor, train_label_tensor, data_to_scale)
train_scl, val_scl = data_scaled

TypeError: scale_data_seq() missing 1 required positional argument: 'data_to_scale'

In [36]:
data[data['total_load'].isna()].iloc[:,15:25]

,total_load,most_recent_forecast_load,most_recent_p10_load,most_recent_p90_load,day-ahead_6pm_forecast_load,measured_&_upscaled_wind,most_recent_forecast_wind,most_recent_p10_wind,most_recent_p90_wind,day-ahead_6pm_forecast_wind
44190,NaN,10175.99,9868.58,10483.39,10330.93,1705.15,1975.13,1334.68,2329.55,1350.19
51754,NaN,6925.21,6680.62,7169.81,7021.81,NaN,1012.41,487.75,1307.83,1054.19
51755,NaN,6933.67,6688.78,7178.56,7030.38,NaN,1008.24,477.29,1306.63,1041.06
51756,NaN,6945.38,6698.08,7192.68,7069.29,NaN,1002.75,470.17,1308.87,1024.51
51757,NaN,6929.03,6682.31,7175.75,7052.65,NaN,987.05,460.69,1287.05,1012.55
...,...,...,...,...,...,...,...,...,...,...
56859,NaN,8567.81,8224.27,8911.36,8478.77,55.05,80.10,14.90,315.00,91.00
56860,NaN,8749.71,8419.35,9080.08,8737.04,60.08,78.60,13.70,318.30,88.40
56861,NaN,8858.99,8523.88,9194.10,8862.54,60.08,76.60,12.50,316.80,83.80
56862,NaN,8946.33,8607.92,9284.75,8949.92,55.03,72.80,11.60,313.50,80.40


In [27]:
len(data)

58360

In [37]:
len(ds_lags)

56826

In [24]:
config["exo_vars"] += new_vars
data_ts, data_tensor, label_tensor = prepare_time_series_data(ds_lags, config["exo_vars"], config["target"], config['src_seq_len'], config['tgt_seq_len'])

In [26]:
len(data_ts)

56786

In [17]:
numerical_known = [
    "index", "month_sin", "month_cos", "day_sin", "day_cos", "hour_sin", "hour_cos", "quarter_hour_sin", "quarter_hour_cos",
    "day-ahead_6pm_forecast_wind", "most_recent_forecast_wind", "day-ahead_6pm_forecast_load", "most_recent_forecast_load", "day-ahead_6pm_forecast_solar", "most_recent_forecast_solar",
]

numerical_unknown = [
    "system_imbalance", "net_regulation_volume", "measured_&_upscaled_solar", "total_load", "measured_&_upscaled_wind",
]

categorical_know = [
    "year", "not_working", "holiday",
]

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
58359    0.0
58360    0.0
58361    0.0
58362    0.0
58363    0.0
Name: not_working, Length: 58364, dtype: float64

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58364 entries, 0 to 58363
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   datetime_utc                  58364 non-null  datetime64[ns]
 1   holiday                       58364 non-null  float64       
 2   not_working                   58364 non-null  float64       
 3   net_regulation_volume         58364 non-null  float64       
 4   system_imbalance              58364 non-null  float64       
 5   alpha                         58364 non-null  float64       
 6   marginal_incremental_price    58364 non-null  float64       
 7   marginal_decremental_price    58364 non-null  float64       
 8   positive_imbalance_price      58364 non-null  float64       
 9   negative_imbalance_price      58364 non-null  float64       
 10  measured_&_upscaled_solar     58364 non-null  float64       
 11  most_recent_forecast_solar  

In [ ]:
from tqdm import tqdm

for epoch in range(0, config['num_epochs']):
    model.train()
    batch_iterator = tqdm(train_dataloader, desc=f"Processing Epoch {epoch:02d}")
    for batch in batch_iterator:
        encoder_input = batch['encoder_input'].to(device) # (batch, seq_len)
        decoder_input = batch['decoder_input'].to(device) # (batch, seq_len)
        encoder_mask = batch['encoder_mask'].to(device) # (batch, 1, 1, seq_len)
        decoder_mask = batch['decoder_mask'].to(device) # (batch, 1, seq_len, seq_len)

        # Run the tensors through the encoder, decoder and the projection layer
        encoder_output = model.encode(encoder_input, encoder_mask) # (batch, seq_len, d_model)
        decoder_output = model.decode(encoder_output, encoder_mask, decoder_input, decoder_mask) # (batch, seq_len, d_model)
        proj_output = model.project(decoder_output) # (batch, seq_len, tgt_vocab_size)

        label = batch['label'].to(device)

        loss = loss_fn(proj_output.view(-1), label.view(-1))
        batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

In [10]:
optimus = build_transformer(num_features = 1, n_tgt = 1, src_seq_len = 8, tgt_seq_len = 2)
batch_size = 4

In [5]:
X = torch.rand(batch_size, 8, 1)
y = torch.rand(batch_size, 2, 1)

In [23]:
y.shape

torch.Size([1, 2, 1])

In [6]:
tgt_mask = causal_mask(2)
src_mask = torch.ones(1, 8, 8).bool()

In [7]:
# optimus.eval()
encoder_output = optimus.encode(X, src_mask)
decoder_output = optimus.decode(encoder_output, src_mask, y, tgt_mask)
proj_output = optimus.project(decoder_output)

In [8]:
print("enc_out:", encoder_output.shape)
print("dec_out:", decoder_output.shape)
print("lin_out:", proj_output.shape)

enc_out: torch.Size([1, 8, 512])
dec_out: torch.Size([1, 2, 512])
lin_out: torch.Size([1, 2, 1])


In [3]:
from torch.utils.data import Dataset, DataLoader

In [5]:
data_raw = pd.read_pickle("./price_15PM_trans.pkl")
data_raw.head()

,net_regulation_volume,system_imbalance,alpha,marginal_incremental_price,marginal_decremental_price,positive_imbalance_price,negative_imbalance_price,holiday,not_working,datetime
datetime,,,,,,,,,,
2021-12-31 23:00:00,-238.403,324.503,2.50,185.55,-58.16,-60.66,-60.66,1,1,2021-12-31 23:00:00
2021-12-31 23:15:00,-274.681,503.440,72.97,154.00,-56.71,-129.68,-129.68,1,1,2021-12-31 23:15:00
2021-12-31 23:30:00,-274.846,248.898,48.62,181.74,-88.24,-136.86,-136.86,1,1,2021-12-31 23:30:00
2021-12-31 23:45:00,-232.667,251.465,8.84,154.00,19.31,10.47,10.47,1,1,2021-12-31 23:45:00
2022-01-01 00:00:00,-50.896,62.864,0.00,154.00,16.65,16.65,16.65,1,1,2022-01-01 00:00:00


In [28]:
data = data_raw
input_seq_len = 8
target_seq_len = 2
target = ["system_imbalance"]
exo_vars = ["net_regulation_volume"]

In [109]:
data_array = data[target + exo_vars].values
data_label = data[target].values

data_tensor = torch.tensor(data_array, dtype=torch.float32)
label_tensor = torch.tensor(data_label, dtype=torch.float32)

data_seq = []

num_obs = len(data_tensor)
max_start_idx = num_obs - input_seq_len - target_seq_len

for start_idx in range(max_start_idx):
    end_idx = start_idx + input_seq_len
    x_input_seq = data_tensor[start_idx:end_idx]
    label_input_seq = label_tensor[start_idx:end_idx]
    
    target_start_idx = end_idx - 1
    target_end_idx = target_start_idx + target_seq_len
    target_dec_seq = label_tensor[target_start_idx:target_end_idx]
    ground_truth = label_tensor[target_start_idx+1:target_end_idx+1]

    data_seq.append({
    "x_input" : x_input_seq,
    "target_decoder" : target_dec_seq,
    "y_true" : ground_truth,
    "target_history" : label_input_seq,
    })


In [110]:
train_ds_index = int(0.7 * len(data_seq))
val_ds_index = int(0.85 * len(data_seq))

train_ds_raw = data_seq[:train_ds_index]
val_ds_raw, test_ds_raw = data_seq[train_ds_index:val_ds_index], data_seq[val_ds_index:]

data_to_scale = {
    "train": train_ds_raw,
    "val": val_ds_raw,
    "test": test_ds_raw
}

In [71]:
for name, dt in data_to_scale.items():
    print(name)

train
val
test


In [118]:
from copy import deepcopy

In [121]:
def scale_data_seq(data_tensor: torch.Tensor, train_ds_index: int, data_to_scale: dict):
    scaled_data = deepcopy(data_to_scale)
    
    enc_scaler = StandardScaler()
    dec_scaler = StandardScaler()

    enc_scaler.fit(data_tensor[:train_ds_index])
    dec_scaler.fit(label_tensor[:train_ds_index])

    for name, dt in data_to_scale.items():
        for ind, obs in enumerate(dt):
            scaled_data[name][ind]['x_input'] = enc_scaler.transform(obs['x_input'])
            scaled_data[name][ind]['target_decoder'] = dec_scaler.transform(obs['target_decoder'])

    return scaled_data['train'], scaled_data['val'], scaled_data['test']

In [122]:
train, val, test = scale_data_seq(data_tensor, train_ds_index, data_to_scale)

In [12]:
lags_dict = {
    "net_regulation_volume": [24*4 - i for i in range(4)],
    "system_imbalance": [24*4 - i for i in range(4)],
}

lagged_difs = {
    "system_imbalance": [24*4 - i for i in range(4)],
}

In [9]:
from importlib import reload

In [15]:
import dataset

reload(dataset)
from dataset import prepare_time_series_data

In [16]:
input_seq_len = 8
target_seq_len = 2
# exo_vars = ["marginal_incremental_price", "marginal_decremental_price"]
exo_vars = []
target = ["system_imbalance"]
data_ts, x_to_scl, y_to_scl = prepare_time_series_data(data_raw, exo_vars, target, input_seq_len, target_seq_len)

In [21]:
x_to_scl[0].shape

torch.Size([8, 1])

In [26]:
dataset = TSDataset(data_ts, input_seq_len, target_seq_len)

In [36]:
dataset[0]["label"].view(-1).unsqueeze(0)

tensor([[33.2830, 99.5400]])

In [27]:
source = dataset[0]["encoder_input"]

In [40]:
dataset[0]["label"].view(-1).unsqueeze(0).type_as(source).to(device)

torch.Size([1, 2])

In [20]:
decoder_input = torch.empty(1,1).fill_(dataset[0]["decoder_input"].view(-1)).type_as(source)

In [ ]:
torch.cat([decoder_input, torch.empty(1,1).type_as(source).fill_(pred).to(device)], dim=1)

In [14]:
import plotly.express as px

# Create an interactive time series plot
fig = px.line(data_raw, x='datetime', y='system_imbalance', title='System Imbalance')
fig.show()

In [15]:
fig = px.line(data_raw[:15*4*24], x='datetime', y='system_imbalance', title='System Imbalance')
fig.show()

In [20]:
data_raw["imbalance_shift"] = abs(data_raw["system_imbalance"].diff(1).fillna(0))
fig = px.line(data_raw[:15*4*24], x='datetime', y='imbalance_shift', line_shape='linear', color_discrete_sequence=["red"], title='System Imbalance')
fig.show()